# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     Camila Garcia Ornelas |   |
| **Fecha**     15 de septiembre |   |
| **Expediente*749725* |   |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Para tener datos “nuevos” donde probar el modelo.. El “test” actúa como ejemplos que el modelo no vio al entrenar, lo cual nos ayuda a simular datos nuevos. 

Usando el train_test_split, sabemos qué tan bien generaliza y si hay overfitting. También nos ayuda a evitar mezclar info de entrenamiento con prueba (sin truco).

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Si usas todo, no tienes con qué evaluar de forma justa (el error en train engaña). Es mejor separar una parte para probar o usa validación cruzada.

Al final, ya elegido el modelo, sí puedes reentrenar con todo para usarlo en producción.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error
import scipy.stats as st

In [30]:
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")
df.columns = df.columns.str.strip()
df = df.drop(columns=['model'], errors='ignore')
y = df['mpg'].astype(float)
X = df.drop(columns=['mpg'])

In [32]:
loo = LeaveOneOut()
mse_list = []


for train_idx, test_idx in loo.split(X):
    X_tr, X_te = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]


    lr = LinearRegression().fit(X_tr, y_tr)
    y_hat = lr.predict(X_te)[0]
    mse_list.append(mean_squared_error([y_te.values[0]], [y_hat]))



mse_array = np.array(mse_list)

mse_mean = mse_array.mean()
mse_std = mse_array.std(ddof=1)
rmse_mean = np.sqrt(mse_mean)

n = len(mse_list)
alpha = 0.05
t_crit = st.t.ppf(1 - alpha/2, df=n-1)

margin_error = t_crit * (mse_std / np.sqrt(n))
IC = (mse_mean - margin_error, mse_mean + margin_error)

print(mse_array)
print()
print(mse_mean)
print()
print(mse_std)
print()
print(rmse_mean)
print()
print(IC)

[5.25885580e+00 2.45399671e+00 2.05504934e+01 4.43291071e-02
 1.58115552e+00 1.01186255e+01 1.63740415e-02 8.08125071e+00
 3.94701835e+01 7.70632772e-01 4.95569423e+00 1.02247302e+01
 4.43012933e+00 4.87812771e-01 6.82384458e+00 6.02750698e-01
 3.68101786e+01 3.17598556e+01 1.06322754e+00 3.27132862e+01
 1.43071737e+01 3.40535465e+00 9.40800765e+00 1.03500398e-04
 9.96243953e+00 1.34122089e+00 1.64800641e-01 2.36002065e+01
 8.31145764e+01 1.02712372e-04 8.78553337e+00 1.75029299e+01]

12.181558006901959

17.34049561034304

3.4902088772596347

(5.929642648176549, 18.433473365627368)


Interpreta.

El mse_array es un vector donde cada número es el error cuadrático para cada observación. Contiene valores muy pequeños que se acercan a cero. Esto indica que el modelo lo predijo casi exacto. Sin embargo, también contiene valores grandes como 1.637e+02 (163.7), 3.271e+01 (32.7), 2.360e+01 (23.6) que quieren decir que hay observaciones difíciles que el modelo lineal no predice bien.

El mse_mean, que en este caso es 12,18, es un promedio de esos errores. El rmse_mean es el error típico de predicción en mpg. Esto quiere decir que cuando se use este modelo para un auto nuevo, espera equivocarte ~3.5 mpg en promedio. El mse_std es la desviación estándar del MSE. Como nos dio 17.34, hay mucha variabilidad entre pliegues; algunos autos dejan errores muy altos. El intervalo de confianza de 95% va de 5.93 a 18.43. Esto quiere decir que probablemente el MSE "real" se encuentre dentro de este rango

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [51]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target


Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [55]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.utils import shuffle

X, y = shuffle(housing.data, housing.target, random_state=42)

print(X.shape, y.shape)
print(housing.feature_names)

(20640, 8) (20640,)
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [83]:
k = 10
indices = np.arange(len(X))
folds = np.array_split(indices, k)

mse_folds = []

for f in range(k):
    test_idx = folds[f]
    train_idx = np.hstack([folds[i] for i in range(k) if i != f])

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx],  y[test_idx]

    lr = LinearRegression().fit(X_train, y_train)
    y_hat = lr.predict(X_test)

    mse = ((y_test - y_hat) ** 2).mean()
    mse_folds.append(mse)
    print("Fold", f+1, "y MSE:", round(mse,3))

mse_folds = np.array(mse_folds)
mse_mean  = mse_folds.mean()
mse_std   = mse_folds.std(ddof=1)
rmse_mean = np.sqrt(mse_mean)

print()
print(mse_folds)
print()
print(mse_mean)
print()
print(mse_std)
print()
print(rmse_mean)

Fold 1 y MSE: 0.559
Fold 2 y MSE: 0.553
Fold 3 y MSE: 0.481
Fold 4 y MSE: 0.574
Fold 5 y MSE: 0.489
Fold 6 y MSE: 0.528
Fold 7 y MSE: 0.551
Fold 8 y MSE: 0.475
Fold 9 y MSE: 0.564
Fold 10 y MSE: 0.55

[0.55900192 0.5531233  0.48116128 0.57420367 0.48897584 0.52838818
 0.55117824 0.47483961 0.56436922 0.55002537]

0.5325266619260605

0.03713364389073527

0.7297442441883735


Interpreta.

El promedio de los errores es aproximadamente 0.53 y la desviación estándar es aproximadamente 0.03. Hay variación entre cada fold. 

El error típico de la regresión lineal es de 0.72. Esto indicaría un error de 72,000 USD. 

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3